# Final

Notebook, en el que se usan todos los modelos

In [1]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing 

In [2]:
import joblib

import torch

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from transformers import BertForSequenceClassification, BertTokenizer

class Models:

    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.bert_topic_model = BERTopic.load("/kaggle/input/bertopic/transformers/default/1/modelo_bertopic",
        embedding_model="all-MiniLM-L6-v2" )

        self.lda_model = LdaModel.load("/kaggle/input/topic_modeling_lda/other/default/1/topic_modeling_lda/model/lda_model_gensim.model")
        self.lda_dictionary = Dictionary.load("/kaggle/input/topic_modeling_lda/other/default/1/topic_modeling_lda/dic/diccionario.dict")
        
        self.svm_sentiment_model = joblib.load("/kaggle/input/ml_sentiment/scikitlearn/default/1/bert_sentiment_model/svm/svm.joblib")
        self.knn_sentiment_model = joblib.load("/kaggle/input/ml_sentiment/scikitlearn/default/1/bert_sentiment_model/knn/knn.joblib")
        self.lr_sentiment_model = joblib.load("/kaggle/input/ml_sentiment/scikitlearn/default/1/bert_sentiment_model/lr/lr.joblib")
        self.nb_sentiment_model = joblib.load("/kaggle/input/ml_sentiment/scikitlearn/default/1/bert_sentiment_model/nb/nb.joblib")
        self.rf_sentiment_model = joblib.load("/kaggle/input/ml_sentiment/scikitlearn/default/1/bert_sentiment_model/rf/rf.joblib")

        self.bert_sentiment_tokenizer = BertTokenizer.from_pretrained("/kaggle/input/bert_sentiment/transformers/default/1/bert_sentiment_model/tokenizer/bert_tokenizer")
        self.bert_sentiment_model = BertForSequenceClassification.from_pretrained("/kaggle/input/bert_sentiment/transformers/default/1/bert_sentiment_model/model/bert_sentiment_model")
        self.id2label_sentiment = {0: "Negativo", 1: "Neutro", 2: "Positivo"}

        self.bert_fake_news_tokenizer = BertTokenizer.from_pretrained("/kaggle/input/bert_fake/transformers/default/1/bert_fake_news/tokenizer/bert_tokenizer")
        self.bert_fake_news_model = BertForSequenceClassification.from_pretrained("/kaggle/input/bert_fake/transformers/default/1/bert_fake_news/model/bert_fake_news_model")
        self.id2label_fake_news = {0: "Falso", 1: "Real"}

        self.svm_fake_news_model = joblib.load("/kaggle/input/ml_fake/scikitlearn/default/1/ml_fake_news/svm/svm.joblib")
        self.knn_fake_news_model = joblib.load("/kaggle/input/ml_fake/scikitlearn/default/1/ml_fake_news/knn/knn.joblib")
        self.lr_fake_news_model = joblib.load("/kaggle/input/ml_fake/scikitlearn/default/1/ml_fake_news/lr/lr.joblib")
        self.nb_fake_news_model = joblib.load("/kaggle/input/ml_fake/scikitlearn/default/1/ml_fake_news/nb/nb.joblib")
        self.rf_fake_news_model = joblib.load("/kaggle/input/ml_fake/scikitlearn/default/1/ml_fake_news/rf/rf.joblib")

    
    def predict_bert_topic(self,doc):
        topic, probs = self.bert_topic_model.transform(doc)
        print(probs)
        if topic != -1:  
            tema = self.bert_topic_model.get_topic(topic[0])  
            palabras = [palabra for palabra, _ in tema]
            print(f"Documento: {doc}\n→ Tópico {topic[0]}: {', '.join(palabras[:10])}\n")
        else:
            print(f"Documento: {doc}\n→ Tópico: No asignado\n")
                
    def __get_topics_lda(self,num_words=10):
        topic_labels = {}
        num_topics = self.lda_model.num_topics
        topics = self.lda_model.show_topics(num_topics=num_topics, formatted=False, num_words=num_words)
        for topic_id, words in topics:
            label = ", ".join([word for word, _ in words])
            topic_labels[topic_id] = label
        return topic_labels

    def __preprocess_lda(self,docs):
        return [word for word in simple_preprocess(docs) if word not in stop_words]

    def __predict_topic_with_label(self,text, topic_labels):
        bow = self.lda_dictionary.doc2bow(self.__preprocess_lda(text))
        topic_dist = self.lda_model.get_document_topics(bow)
        if not topic_dist:
            return None, None
        main_topic_id = sorted(topic_dist, key=lambda x: -x[1])[0][0]
        return main_topic_id, topic_labels.get(main_topic_id, "Tema desconocido")

    def predict_lda_topic(self, texto):
        topic_labels = self.__get_topics_lda()
        
        topic_id, topic_name = self.__predict_topic_with_label(texto, topic_labels)
        
        print(f"Tema predicho: {topic_id} - {topic_name}")

    def __predict_bert(self, texto, model, tokenizer, max_lenght, id2label):
        inputs = tokenizer(
                texto,
                add_special_tokens=True,
                max_length=max_lenght,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
    
        # Mover los tensores al dispositivo (CPU o GPU)
        input_ids = inputs["input_ids"].to(self.device)
        attention_mask = inputs["attention_mask"].to(self.device)
    
        # Desactivar gradientes para inferencia
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred_id = torch.argmax(logits, dim=1).item()
    
        return id2label[pred_id]
        #return pred_id

    def predict_bert_sentiment(self, text):
        return self.__predict_bert(text, self.bert_sentiment_model, self.bert_sentiment_tokenizer, 128,self.id2label_sentiment)

    def predict_ml_sentiment(self, text, slc_model = 'svm'):
        if(slc_model == 'svm'):
            pred = self.id2label_sentiment[self.svm_sentiment_model.predict([text])[0]]
        elif(slc_model == 'knn'):
            pred = self.id2label_sentiment[self.knn_sentiment_model.predict([text])[0]]
        elif(slc_model == 'lr'):
            pred = self.id2label_sentiment[self.lr_sentiment_model.predict([text])[0]]
        elif(slc_model == 'nb'):
            pred = self.id2label_sentiment[self.nb_sentiment_model.predict([text])[0]]
        elif(slc_model == 'rf'):
            pred = self.id2label_sentiment[self.rf_sentiment_model.predict([text])[0]]
        return pred

    def predict_bert_fake_news(self, text):
        return self.__predict_bert(text, self.bert_fake_news_model, self.bert_fake_news_tokenizer, 512,self.id2label_fake_news)

    def predict_ml_fake_news(self, text, slc_model = 'svm'):
        if(slc_model == 'svm'):
            pred = self.id2label_fake_news[self.svm_fake_news_model.predict([text])[0]]
        elif(slc_model == 'knn'):
            pred = self.id2label_fake_news[self.knn_fake_news_model.predict([text])[0]]
        elif(slc_model == 'lr'):
            pred = self.id2label_fake_news[self.lr_fake_news_model.predict([text])[0]]
        elif(slc_model == 'nb'):
            pred = self.id2label_fake_news[self.nb_fake_news_model.predict([text])[0]]
        elif(slc_model == 'rf'):
            pred = self.id2label_fake_news[self.rf_fake_news_model.predict([text])[0]]
        return pred

2025-07-13 09:48:09.541196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752400089.767598      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752400089.834385      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
import time

model = Models()

start = time.time()

texto = "A controversial new study published by the non-peer-reviewed journal Global Wellness Watch claims that major coffee brands have been embedding microscopic tracking chips inside coffee beans since 2021. The alleged goal: to monitor consumer drinking habits and location data in real time."

model.predict_bert_topic(texto)
end = time.time()

elapsed_ms = (end - start) * 1000  # convertir a milisegundos
print(f"Tiempo transcurrido: {elapsed_ms:.2f} ms")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-07-13 09:53:12,485 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-07-13 09:53:12,495 - BERTopic - Dimensionality - Completed ✓
2025-07-13 09:53:12,496 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-07-13 09:53:12,498 - BERTopic - Cluster - Completed ✓


[0.89869101]
Documento: A controversial new study published by the non-peer-reviewed journal Global Wellness Watch claims that major coffee brands have been embedding microscopic tracking chips inside coffee beans since 2021. The alleged goal: to monitor consumer drinking habits and location data in real time.
→ Tópico 33: coffee, tea, starbucks, cup, espresso, iced, cups, good, having, baristas

Tiempo transcurrido: 57.63 ms
